This notebook extracts the coastline polygons from ``planet-complete.db`` and saves them in the multipolygon binary format used by the r2r system. The output file is ``poly.bin.gz`` (used by LocationScraper).

In [1]:
from pyspatialite import dbapi2 as db
import shapely
import struct

In [2]:
conn = db.connect('/home/mgi/raw/planet-complete.db')

In [3]:
cursor = conn.cursor()

In [4]:
for row in cursor.execute('select sqlite_version(), spatialite_version()'):
    print "Sqlite version %s, Spatialite version %s" % (row[0], row[1])

Sqlite version 3.8.2, Spatialite version 4.1.1


In [5]:
import shapely.geometry as geometry

In [6]:
from shapely import wkt

In [7]:
shapes = []
response = cursor.execute('select AsText(Geometry) from land_polygons')
for row in response:
    shapes.append(wkt.loads(row[0]))

In [8]:
len(shapes)

581435

In [9]:
class Ring:
    def __init__(self, outer, spatialiteRing, children):
        self.outer = outer
        self.ring = spatialiteRing
        self.children = children
        
print "Constructing rings"
outer_rings = []
inner_rings = []
for poly in [x for x in shapes if x.type == "Polygon"]:
    children = [Ring(False, ring, []) for ring in poly.interiors]
    inner_rings.extend(children)
    outer_rings.append(Ring(True, poly.exterior, children))

def resolve():    
    print "Resolving internal islands (on %d lakes/seas)" % len(inner_rings)
    remove = set()
    pct = 0
    for i, coastline in enumerate(outer_rings):
        if (100 * i / len(outer_rings)) != pct:
            pct = 100 * i / len(outer_rings)
            print "%d%%" % pct
        cminx, cminy, cmaxx, cmaxy = coastline.ring.bounds
        for lake in inner_rings:
            minx, miny, maxx, maxy = lake.ring.bounds
            if cminx > minx and cmaxx < maxx and cminy > miny and cmaxy < maxy:
                contains = True
                for p in coastline.ring.coords:
                    if not lake.ring.contains(geometry.Point(p[0], p[1])):
                        contains = False
                        break
                if contains:
                    lake.children.append(coastline)
                    remove.add(coastline)
                    break
                
    print "Done (%d islands resolved)" % len(remove)
    outer_rings = [ring for ring in outer_rings if ring not in remove]

Constructing rings


In [10]:
import struct
import gzip

assert struct.calcsize("=i") == 4
assert struct.calcsize("=ff") == 8

def dump_coords(fp, coords):
    fp.write(struct.pack("=i", len(coords)))
    for lng, lat in coords:
        fp.write(struct.pack("=ff", lat, lng))
        
def dump_ring(fp, ring):
    dump_coords(fp, ring.ring.coords)
    fp.write(struct.pack("=i", len(ring.children)))
    for child in ring.children:
        dump_coords(fp, child.ring.coords)
        fp.write(struct.pack("=i", 0))  # zero grandchildren
        
def dump_ringset(filename, rings):
    with gzip.open(filename, "wb") as fp:
        fp.write(struct.pack("=i", len(rings)))
        for ring in rings:
            dump_ring(fp, ring)

In [11]:
dump_ringset("/mnt/src/Working/Data/OSM/Processed/poly.bin.gz", outer_rings)